# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import torch
import numpy as np
from agent.ppo import PPO, RolloutBuffer
from collections import deque

In [2]:
env = UnityEnvironment(file_name='Reacher_Linux/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726624e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [4]:
def play_episode(env, brain_name, agent, train_buffer=None, max_t=1000, trajectory_length=16):
    env_info = env.reset(train_mode=True if train_buffer else False)[brain_name]
    states = env_info.vector_observations
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    for t in range(max_t):
        actions, action_log_probs = agent.act(states)
        env_info = env.step(actions)[brain_name]

        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done

        scores += rewards                         # update the score (for each agent)

        if train_buffer:
            train_buffer.states.append(states)
            train_buffer.actions.append(actions)
            train_buffer.logprobs.append(action_log_probs)
            train_buffer.rewards.append(rewards)
            train_buffer.is_terminals.append(dones)
            if (t+1) % trajectory_length == 0:
                agent.update(train_buffer)
                train_buffer.clear()

        states = next_states
        if np.any(dones):
            break
    return scores


def train_agent(agent, env, brain_name, num_episodes=1000, max_t=1000, trajectory_length=16):
    buffer = RolloutBuffer()
    scores_deque = deque(maxlen=100)
    scores_history = []
    for i_episode in range(1, num_episodes+1):
        scores = play_episode(env, brain_name, agent, train_buffer=buffer, max_t=max_t, trajectory_length=trajectory_length)
        avg_score = np.mean(scores)
        scores_history.append(avg_score)
        print('episode :{}\tscore: {}'.format(i_episode, avg_score))

    return scores_history

In [5]:
agent = PPO(state_size, action_size, 1e-2, 1e-3, 0.99, 8, 0.1, 0.05)
history = train_agent(agent, env, brain_name, num_episodes=50, trajectory_length=32)

episode :1	score: 0.49349998896941544
episode :2	score: 0.45649998979642986
episode :3	score: 0.3574999920092523
episode :4	score: 0.4669999895617366
episode :5	score: 0.7354999835602939
episode :6	score: 0.5514999876730144
episode :7	score: 0.5194999883882702
episode :8	score: 0.5994999866001308
episode :9	score: 0.526499988231808
episode :10	score: 0.5134999885223805
episode :11	score: 0.685499984677881
episode :12	score: 0.6219999860972166
episode :13	score: 0.5329999880865216
episode :14	score: 0.541999987885356
episode :15	score: 0.38299999143928287
episode :16	score: 0.6369999857619405
episode :17	score: 0.3854999913834035
episode :18	score: 0.4759999893605709
episode :19	score: 0.48049998925998805
episode :20	score: 0.43749999022111297


[0.49349998896941544,
 0.45649998979642986,
 0.3574999920092523,
 0.4669999895617366,
 0.7354999835602939,
 0.5514999876730144,
 0.5194999883882702,
 0.5994999866001308,
 0.526499988231808,
 0.5134999885223805,
 0.685499984677881,
 0.6219999860972166,
 0.5329999880865216,
 0.541999987885356,
 0.38299999143928287,
 0.6369999857619405,
 0.3854999913834035,
 0.4759999893605709,
 0.48049998925998805,
 0.43749999022111297]

When finished, you can close the environment.

In [6]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```